## Pricing by PDE

In [2]:
import numpy as np

### Terminal conditions functions

In [3]:
def call_fixed_strike (x) :
    """
    Give the terminal condition as presented in the pdf
    
    Parameters :
    - x : real number
    
    Return :
    Terminal condition as presented in the pdf
    """
    if -x > 0 :
        return -x
    else :
        return 0
    

    
def call_float_strike (x)  :
    """
    Give the terminal condition as presented in the pdf
    
    Parameters :
    - x : real number
    
    Give the terminal condition as presented in the pdf
    """
    if -(1+x) > 0 :
        return -(1+x)
    else :
        return 0   
     

### Function sigma, b and the lower bond function

In [9]:
def sigma_function (x,sigma):
    """
    Give the function sigma as presented in the PDF
    
    Parameters :
    - x : float number
    - sigma : constant parameter, volatility of the model
    
    Return :
    x*sigma
    """
    return x*sigma

def b(r,x,T) :
    """
    Give the function b as presented in the PDF in the case of a fixed strike
    
    Parameters :
    - x : float number
    - r : constant number, interest rate
    - T : constant number, maximum time step
    
    Return:
    -(r*x +(1/T))
    """
    return -(r*x +(1/T))

    

def lower_bound_fixed_call (alpha,r, T, t) :
    """
    Give the lower bound as presented in the PDF in the case of a fixed strike
    
    Parameters :
    - alpha : constant number, initial x's step
    - r :constant number, interet rate
    - T : constant number, maximum time step
    - t: constant number, current time step
    
    Return :
    (np.exp(r*(T-t))-1)/(r*T) - alpha
    """
    
    return np.exp(-r*(T-t))*((np.exp(r*(T-t))-1)/(r*T) - alpha)

def lower_bound_float_call (alpha, r, T, t) :
    
    """
    Give the lower bound as presented in the PDF in the case of a floating strike
    
    Parameters :
    - alpha : constant number, initial x's step
    - r :constant number, interet rate
    - T : constant number, maximum time step
    - t: constant number, current time step
    
    Return :
    np.exp(-r*(T-t))((np.exp(r*(T-t))-1)/(r*T) - np.exp(r*(T-t)) - alpha)
    """
    return np.exp(-r*(T-t))*((np.exp(r*(T-t))-1)/(r*T) - np.exp(r*(T-t)) - alpha)




### Main program for fixed call Asian option

In [5]:
def fixed_call (T,m,r,sigma,S0,K,alpha,beta,l) :
    """
    Compute the price of an Asian call option with fixed strike K
    
    Parameters :
    T : constant number, maximum time step
    m : number of time step
    r : interest rate
    sigma : volatility
    S0 : initial price of the asset
    alpha : initial x's step
    beta : last x's step
    K : fixed strike
    l : number of x's step (-1)
    
    
    Return :
    The price of a call option using an interpolation method
    
    """
    h = T/m
    delta = (beta-alpha)/(l+1)

    # Define the mesh
    t = np.arange(0,T+h,h)
    x = np.arange(alpha,beta+delta,delta)

    # Initialize the matrix f as presented in the PDF
    dim_line = len(x)
    dim_col = len(t)
    F = np.zeros((dim_line,dim_col))

    # Define the boundary condition
    for time in range(len(t)) :
        F[0,time] =  lower_bound_fixed_call (alpha,r, T, t[time])
    

    # Define the terminal condition
    for pos in range(len(x)) :
        F[pos,-1] = call_fixed_strike(x[pos])


    # Define the matrix A as presented in the PDF
    A = np.zeros((dim_line-2,dim_line-2))
    for i in range(1,dim_line-1) :
    
        if i == 1 :
            A[i-1,i-1] = -(sigma_function(x[i],sigma)**2)/(delta**2)
            A[i-1,i] = (sigma_function(x[i],sigma)**2)/(2*delta**2) + (b(r,x[i],T)/(2*delta))
    
        elif i == dim_line-2 :
            A[i-1,i-1] = -(sigma_function(x[i],sigma)**2)/(delta**2)
            A[i-1,i-2] = (sigma_function(x[i],sigma)**2)/(2*delta**2) - (b(r,x[i],T)/(2*delta))
    
        else :
    
            A[i-1,i-2] = (sigma_function(x[i],sigma)**2)/(2*delta**2) - (b(r,x[i],T)/(2*delta))
            A[i-1,i-1] = -(sigma_function(x[i],sigma)**2)/(delta**2)
            A[i-1,i] = (sigma_function(x[i],sigma)**2)/(2*delta**2) + (b(r,x[i],T)/(2*delta))
    
    # Get the time step necessary to update the matrix f
    time_step = np.flip(np.arange(0,m,1))
    
    # Definition of X as written in the PDF
    Id = np.eye(dim_line-2)
    X = Id - 0.5*h*A

    # extraction of f^{m} as mentioned in the PDF
    Ftmp = F[1:-1,-1]


    for time in time_step :
    
        # Definition of y as written the PDF
        y = np.dot(Id+0.5*h*A,Ftmp)
        y[0] = y[0] + h * 0.5*(((sigma_function(x[1],sigma)**2)/(2*delta**2) - (b(r,x[1],T)/(2*delta))) * (lower_bound_fixed_call(alpha,r, T, t[time])+lower_bound_fixed_call(alpha,r, T, t[time+1])))
    
        Ftmp = np.linalg.solve(X,y)
    
        # Update of the matrix f
        F[1:-1,time] = Ftmp
        
    return(round(np.interp(K/S0,x,S0*F[:,0]),2))
    

In [6]:
# Initialize parameters
T = 1
#m = 200
r = 0.1
S0 = 100
K = 100
alpha = 0
beta = (K/S0)*2
#l = 200

# List of volatility and step to test
sigmas = [0.05,0.2,0.3]
step = [50,100,200,300,1000]

for sigma in sigmas :
    for elem in step :
        m = elem
        l = elem
        print("Number of time steps :",m)
        print("Number of x step :",l+1)
        print("sigma :",sigma)
        print("Price of the Asian call with fixed strike :",fixed_call (T,m,r,sigma,S0,K,alpha,beta,l))
        print()

Number of time steps : 50
Number of x step : 51
sigma : 0.05
Price of the Asian call with fixed strike : 4.79

Number of time steps : 100
Number of x step : 101
sigma : 0.05
Price of the Asian call with fixed strike : 4.24

Number of time steps : 200
Number of x step : 201
sigma : 0.05
Price of the Asian call with fixed strike : 4.29

Number of time steps : 300
Number of x step : 301
sigma : 0.05
Price of the Asian call with fixed strike : 4.48

Number of time steps : 1000
Number of x step : 1001
sigma : 0.05
Price of the Asian call with fixed strike : 4.71

Number of time steps : 50
Number of x step : 51
sigma : 0.2
Price of the Asian call with fixed strike : 6.93

Number of time steps : 100
Number of x step : 101
sigma : 0.2
Price of the Asian call with fixed strike : 6.96

Number of time steps : 200
Number of x step : 201
sigma : 0.2
Price of the Asian call with fixed strike : 7.02

Number of time steps : 300
Number of x step : 301
sigma : 0.2
Price of the Asian call with fixed stri

In [10]:
def float_call (T,m,r,sigma,S0,alpha,beta,l) :
    """
    Compute the price of an Asian call option with floating strike
    
    Parameters :
    T : constant number, maximum time step
    m : number of time step
    r : interest rate
    sigma : volatility
    S0 : initial price of the asset
    alpha : initial x's step
    beta : last x's step
    l : nnumber of x's step (-1)
    
    
    Return :
    The price of a call option using an interpolation method
    
    """
    h = T/m
    delta = (beta-alpha)/(l+1)

    # Define the mesh
    t = np.arange(0,T+h,h)
    x = np.arange(alpha,beta+delta,delta)

    # Initialize the matrix f as presented in the PDF
    dim_line = len(x)
    dim_col = len(t)
    F = np.zeros((dim_line,dim_col))

    # Define the boundary condition
    for time in range(len(t)) :
        F[0,time] =  lower_bound_float_call (alpha,r, T, t[time])
    

    # Define the terminal condition
    for pos in range(len(x)) :
        F[pos,-1] = call_float_strike(x[pos])


    # Define the matrix A as presented in the PDF
    A = np.zeros((dim_line-2,dim_line-2))
    for i in range(1,dim_line-1) :
    
        if i == 1 :
            A[i-1,i-1] = -(sigma_function(x[i],sigma)**2)/(delta**2)
            A[i-1,i] = (sigma_function(x[i],sigma)**2)/(2*delta**2) + (b(r,x[i],T)/(2*delta))
    
        elif i == dim_line-2 :
            A[i-1,i-1] = -(sigma_function(x[i],sigma)**2)/(delta**2)
            A[i-1,i-2] = (sigma_function(x[i],sigma)**2)/(2*delta**2) - (b(r,x[i],T)/(2*delta))
    
        else :
    
            A[i-1,i-2] = (sigma_function(x[i],sigma)**2)/(2*delta**2) - (b(r,x[i],T)/(2*delta))
            A[i-1,i-1] = -(sigma_function(x[i],sigma)**2)/(delta**2)
            A[i-1,i] = (sigma_function(x[i],sigma)**2)/(2*delta**2) + (b(r,x[i],T)/(2*delta))
    
    time_step = np.flip(np.arange(0,m,1))
    
    # Definition of X as written in the PDF
    Id = np.eye(dim_line-2)
    X = Id - 0.5*h*A

    # extraction of f^{m} as mentioned in the PDF
    Ftmp = F[1:-1,-1]


    for time in time_step :
    
        # Definition of y as written the PDF
        y = np.dot(Id+0.5*h*A,Ftmp)
        y[0] = y[0] + h * 0.5*(((sigma_function(x[1],sigma)**2)/(2*delta**2) - (b(r,x[1],T)/(2*delta))) * (lower_bound_fixed_call(alpha,r, T, t[time])+lower_bound_fixed_call(alpha,r, T, t[time+1])))
    
        Ftmp = np.linalg.solve(X,y)
    
        # Update of the matrix f
        F[1:-1,time] = Ftmp
        
    return(round(np.interp(0,x,S0*F[:,0]),2))
 

In [11]:
# Initialize parameters
T = 1
#m = 200
r = 0.1
S0 = 100
alpha = -10
beta = 10
#l = 200

# List of volatility and step to test
sigmas = [0.05,0.2,0.3]
step = [50,100,200,300,1000]


for sigma in sigmas :
    for elem in step :
        m = elem
        l = elem
        print("Number of time steps :",m)
        print("Number of x step :",l+1)
        print("sigma :",sigma)
        print("Price of the Asian call with floating strike :",float_call (T,m,r,sigma,S0,alpha,beta,l))
        print()

Number of time steps : 50
Number of x step : 51
sigma : 0.05
Price of the Asian call with fixed strike : 9.2

Number of time steps : 100
Number of x step : 101
sigma : 0.05
Price of the Asian call with fixed strike : 5.79

Number of time steps : 200
Number of x step : 201
sigma : 0.05
Price of the Asian call with fixed strike : 3.07

Number of time steps : 300
Number of x step : 301
sigma : 0.05
Price of the Asian call with fixed strike : 2.07

Number of time steps : 1000
Number of x step : 1001
sigma : 0.05
Price of the Asian call with fixed strike : 0.55

Number of time steps : 50
Number of x step : 51
sigma : 0.2
Price of the Asian call with fixed strike : 9.79

Number of time steps : 100
Number of x step : 101
sigma : 0.2
Price of the Asian call with fixed strike : 6.54

Number of time steps : 200
Number of x step : 201
sigma : 0.2
Price of the Asian call with fixed strike : 4.13

Number of time steps : 300
Number of x step : 301
sigma : 0.2
Price of the Asian call with fixed strik